In [1]:
cd ../../apps

e:\kaggle\pytorch-book\apps


In [2]:
from torchvision.utils import save_image
import torch
import time
from matplotlib import pyplot as plt 

from tools.file import mkdir
from utils.torch_loader_all import Loader
from tools.toml import load_option
from app import  init, mask_op, array2image

In [3]:
opt = load_option('../origin/train-512.toml')
print(opt)
loader = Loader(**opt)

{'root': 'E:/kaggle/datasets/tune', 'mask': 'D:/kaggle/dataset/mask/testing_mask_dataset', 'fine_size': 512, 'batch_size': 1}


In [4]:
model_name = 'CSA-512'
beta = 1
base_opt = load_option('../options/base512.toml')
model_opt = load_option('../options/train-new.toml')
model = init(model_name, beta, model_opt, base_opt)

initialize network with normal
initialize network with normal
initialize network with normal
initialize network with normal
---------- Networks initialized -------------
UnetGeneratorCSA(
  (model): UnetSkipConnectionBlock_3(
    (model): Sequential(
      (0): Conv2d(6, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
      (1): UnetSkipConnectionBlock_3(
        (model): Sequential(
          (0): LeakyReLU(negative_slope=0.2, inplace=True)
          (1): Conv2d(64, 64, kernel_size=(4, 4), stride=(2, 2), padding=(3, 3), dilation=(2, 2))
          (2): InstanceNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=False)
          (3): LeakyReLU(negative_slope=0.2, inplace=True)
          (4): Conv2d(64, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
          (5): InstanceNorm2d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=False)
          (6): UnetSkipConnectionBlock_3(
            (model): Sequential(
              (0): LeakyReLU(negat

In [5]:
# 超参数设定
## 固定参数
epochs = 500
display_freq = 49
save_epoch_freq = 1

## 模型参数
alpha = 1
beta = 1


model_name = f'CSA-crop-{alpha}-{beta}'
image_save_dir = model.save_dir / 'images'
mkdir(image_save_dir)

In [6]:
# 训练阶段
start_epoch = 0
total_steps = 0
iter_start_time = time.time()
for epoch in range(start_epoch, epochs):
    epoch_start_time = time.time()
    epoch_iter = 0
    epoch_iter = 0
    # 初始化数据集
    trainset = loader.trainset() # 训练集
    maskset = loader.maskset() # mask 数据集
    for (image, _), mask in zip(trainset, maskset):
        mask = mask_op(mask)
        total_steps += model.batch_size
        epoch_iter += model.batch_size
        # it not only sets the input data with mask,
        #  but also sets the latent mask.
        model.set_input(image, mask)
        model.set_gt_latent()
        model.optimize_parameters()
        if total_steps % display_freq == 0:
            real_A, real_B, fake_B = model.get_current_visuals()
            # real_A=input, real_B=ground truth fake_b=output
            pic = (torch.cat([real_A, real_B, fake_B], dim=0) + 1) / 2.0
            image_name = f"epoch{epoch}-{total_steps}-{alpha}.png"
            save_image(pic, image_save_dir/image_name, ncol=1)
        if total_steps % 100 == 0:
            errors = model.get_current_errors()
            t = (time.time() - iter_start_time) / model.batch_size
            print(
                f"Epoch/total_steps/alpha-beta: {epoch}/{total_steps}/{alpha}-{beta}", dict(errors))
    if epoch % save_epoch_freq == 0:
        print(f'保存模型 Epoch {epoch}, iters {total_steps} 在 {model.save_dir}')
        model.save(epoch)
    print(
        f'Epoch/Epochs {epoch}/{epochs-1} 花费时间：{time.time() - epoch_start_time}s')
    model.update_learning_rate()

Epoch/total_steps/alpha-beta: 0/100/1-1 {'G_GAN': 4.890397071838379, 'G_L1': 55.89939880371094, 'D': 1.0857429504394531, 'F': 0.17773281037807465}
Epoch/total_steps/alpha-beta: 0/200/1-1 {'G_GAN': 5.800795078277588, 'G_L1': 78.20355987548828, 'D': 0.6086318492889404, 'F': 0.06901766359806061}
Epoch/total_steps/alpha-beta: 0/300/1-1 {'G_GAN': 6.616123199462891, 'G_L1': 65.0513916015625, 'D': 0.4572766125202179, 'F': 0.04627228528261185}
Epoch/total_steps/alpha-beta: 0/400/1-1 {'G_GAN': 6.499786853790283, 'G_L1': 45.429290771484375, 'D': 0.5360347032546997, 'F': 0.0745968446135521}
Epoch/total_steps/alpha-beta: 0/500/1-1 {'G_GAN': 6.127684593200684, 'G_L1': 70.71878051757812, 'D': 0.5126592516899109, 'F': 0.0815393254160881}
Epoch/total_steps/alpha-beta: 0/600/1-1 {'G_GAN': 5.6323652267456055, 'G_L1': 29.22673797607422, 'D': 0.838119387626648, 'F': 0.04744487255811691}
Epoch/total_steps/alpha-beta: 0/700/1-1 {'G_GAN': 6.599039077758789, 'G_L1': 43.16686248779297, 'D': 0.5080881118774414,

KeyboardInterrupt: 